<a href="https://colab.research.google.com/github/chris-kehl/BattleTankImageRecognition/blob/master/ArmyTankClassifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

The purpose of this project is to use transfer learning for large image classification to detect three different sets of main battle tanks which are used by the U.S military, the Russian military, and the German military forces.

The first step is to arrange our data into folders. We create a train folder, a validation  folder, and a test folder.

In [0]:
import os, shutil

In [0]:


original_abrams_dataset_dir = '/content/drive/My Drive/data_files/data/abrams'
original_leopard_dataset_dir = '/content/drive/My Drive/data_files/data/leopard'
original_t90_dataset_dir = '/content/drive/My Drive/data_files/data/t90'

base_dir = '/content/drive/My Drive/data_files/tanks'
os.mkdir(base_dir)

train_dir = os.path.join(base_dir, 'train')
os.mkdir(train_dir)
validation_dir = os.path.join(base_dir, 'validation')
os.mkdir(validation_dir)
test_dir = os.path.join(base_dir, 'test')
os.mkdir(test_dir)

In the next block of code we are going to build out train, validation, and test folders. Each folder will consist of a subfolder for each tank image. Each subfolder will contain the pictures of the photos of the tanks folder label.

In [0]:
train_us_dir = os.path.join(train_dir, 'US')
os.mkdir(train_us_dir)

train_foreign_dir = os.path.join(train_dir, 'Foreign')
os.mkdir(train_foreign_dir)

validation_us_dir = os.path.join(validation_dir, 'US')
os.mkdir(validation_us_dir)

validation_foreign_dir = os.path.join(validation_dir, 'Foreign')
os.mkdir(validation_foreign_dir)

test_us_dir = os.path.join(test_dir, 'US')
os.mkdir(test_us_dir)

test_foreign_dir = os.path.join(test_dir, 'Foreign')
os.mkdir(test_foreign_dir)


fnames = ['abrams_{}.jpg'.format(i) for i in range(60)]
for fname in fnames:
  scr = os.path.join(original_abrams_dataset_dir, fname)
  dst = os.path.join(train_us_dir, fname)
  shutil.copyfile(scr, dst)

fnames = ['abrams_{}.jpg'.format(i) for i in range(60, 80)]
for fname in fnames:
  scr = os.path.join(original_abrams_dataset_dir, fname)
  dst = os.path.join(validation_us_dir, fname)
  shutil.copyfile(scr, dst)

fnames = ['abrams_{}.jpg'.format(i) for i in range(80, 100)]
for fname in fnames:
  scr = os.path.join(original_abrams_dataset_dir, fname)
  dst = os.path.join(test_us_dir, fname)
  shutil.copyfile(scr, dst)

fnames = ['leopard_{}.jpg'.format(i) for i in range(60)]
for fname in fnames:
  scr = os.path.join(original_leopard_dataset_dir, fname)
  dst = os.path.join(train_foreign_dir, fname)
  shutil.copyfile(scr, dst)

fnames = ['leopard_{}.jpg'.format(i) for i in range(60, 80)]
for fname in fnames:
  scr = os.path.join(original_leopard_dataset_dir, fname)
  dst = os.path.join(validation_foreign_dir, fname)
  shutil.copyfile(scr, dst)

fnames = ['leopard_{}.jpg'.format(i) for i in range(80, 100)]
for fname in fnames:
  scr = os.path.join(original_leopard_dataset_dir, fname)
  dst = os.path.join(test_foreign_dir, fname)
  shutil.copyfile(scr, dst)

fnames = ['t90_{}.jpg'.format(i) for i in range(60)]
for fname in fnames:
  scr = os.path.join(original_t90_dataset_dir, fname)
  dst = os.path.join(train_foreign_dir, fname)
  shutil.copyfile(scr, dst)

fnames = ['t90_{}.jpg'.format(i) for i in range(60, 80)]
for fname in fnames:
  scr = os.path.join(original_t90_dataset_dir, fname)
  dst = os.path.join(validation_foreign_dir, fname)
  shutil.copyfile(scr, dst)

fnames = ['t90_{}.jpg'.format(i) for i in range(80, 100)]
for fname in fnames:
  scr = os.path.join(original_t90_dataset_dir, fname)
  dst = os.path.join(test_foreign_dir, fname)
  shutil.copyfile(scr, dst)



We will take a look at our folders

In [0]:
print('total training us images:', len(os.listdir(train_us_dir)))
print('total training foreign images:', len(os.listdir(train_foreign_dir)))
print('total validation us images:', len(os.listdir(validation_us_dir)))
print('total validation foreign images:', len(os.listdir(validation_foreign_dir)))
print('total test us images:', len(os.listdir(test_us_dir)))
print('total test foreign images:', len(os.listdir(test_foreign_dir)))


Building our network

In [0]:
from keras import layers
from keras import models

model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu',
                        input_shape=(150, 150, 3)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Flatten())
model.add(layers.Dense(512, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))

model.summary()

Configure the model for training

In [0]:
from keras import optimizers

model.compile(loss='binary_crossentropy',
              optimizer=optimizers.RMSprop(lr=1e-4),
              metrics=['acc'])


In [0]:
from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(150, 150),
    batch_size=1,
    class_mode='binary')

validation_generator = test_datagen.flow_from_directory(
    validation_dir,
    target_size=(150, 150),
    batch_size=1,
    class_mode='binary')



In [0]:
for data_batch, labels_batch in train_generator:
  print('data batch shape:', data_batch.shape)
  print('label batch shape:', labels_batch.shape)
  break

In [0]:
history = model.fit_generator(
    train_generator,
    steps_per_epoch=100,
    epochs=3,
    validation_data=validation_generator,
    validation_steps=50)

model.save('tanks_1.h5')

Lets plot the training and validation accuracy to see if there is any overfitting going on.  Looking at the validaion accuaracy we are going to have to do more tweaking of our model to bring up the validation accuracy percentages.

In [0]:
import matplotlib.pyplot as plt

acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(1, len(acc) + 1)

plt.plot(epochs, acc, 'bo', label='Training acc')
plt.plot(epochs, val_acc, 'b', label = 'Validation acc')
plt.title('Training and Validation Accuracy')
plt.legend()

plt.figure()

plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and Validation loss')
plt.legend()

plt.show()

Add data augmentation

In [0]:
datagen = ImageDataGenerator(
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest')

Looking at the images that we just augmented

In [0]:
from keras.preprocessing import image

fnames = [os.path.join(train_us_dir, fname) for
          fname in os.listdir(train_us_dir)]

img_path = fnames[3]

img = image.load_img(img_path, target_size=(150, 150))

x = image.img_to_array(img)
x = x.reshape((1,) + x.shape)

i = 0

for batch in datagen.flow(x, batch_size=1):
  plt.figure(i)
  imgplot = plt.imshow(image.array_to_img(batch[0]))
  i += 1
  if i % 4 == 0:
    break

plt.show()


To improve our model, we will define a new convolutional network model that will include dropout.

In [0]:
model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu',
          input_shape=(150, 150, 3)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Flatten())
model.add(layers.Dropout(0.5))
model.add(layers.Dense(512, activation='relu'))
model.add(layers.Dense(1, activation='softmax'))

model.compile(loss='binary_crossentropy',
              optimizer=optimizers.RMSprop(lr=1e-4),
              metrics=['acc'])

Train the model network using data augmentation and dropout.

In [0]:
train_datagen = ImageDataGenerator(
    rescale=1.255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest')

test_generator = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='binary')

validation_generator = test_datagen.flow_from_directory(
    validation_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='binary')

history = model.fit_generator(
    train_generator,
    steps_per_epoch=100,
    epochs=3,
    validation_data=validation_generator,
    validation_steps=50)

model.save('tanks_2h5')

Use the VGG16 architecture to use a pretrained CNN


In [0]:
from keras.applications import VGG16

conv_base = VGG16(weights='imagenet',
                  include_top=False,
                  input_shape=(150, 150, 3))

The above Keras CNN resulted in 72% accuracy without data augmentation and with data augmentation it was worse. I am going to implement fast feature extraction without data augmentation to see how that model performs.


In [0]:
import numpy as np
from keras.preprocessing.image import ImageDataGenerator

base_dir = '/content/drive/My Drive/data_files/tanks'
train_dir = os.path.join(base_dir, 'train')
validation_dir = os.path.join(base_dir, 'validation')
test_dir = os.path.join(base_dir, 'test')

datagen = ImageDataGenerator(rescale=1./255)
batch_size = 5

def extract_features(directory, sample_count):
  features = np.zeros(shape=(sample_count, 4, 4, 512))
  labels = np.zeros(shape=(sample_count))
  generator = datagen.flow_from_directory(
      directory,
      target_size=(150, 150),
      batch_size=batch_size,
      class_mode='binary')
  
  i = 0
  for inputs_batch, labels_batch in generator:
    features_batch = conv_base.predict(inputs_batch)
    features[i * batch_size : (i + 1) * batch_size] = features_batch
    labels[i * batch_size : (i + 1) * batch_size] = labels_batch
    i + 1
    if i * batch_size >= sample_count:
      break
    return features, labels

train_features, train_labels = extract_features(train_dir, 180)
validation_features, validation_labels = extract_features(validation_dir, 60)
test_features, test_labels = extract_features(test_dir, 60)

The current shape of extracted features are (samples, 4, 4, 512) I will flatten the features in order to feed the features into the densly connected classifier.


In [0]:
train_features = np.reshape(train_features, (180, 4 * 4 * 512))
validation_features = np.reshape(validation_features, (60, 4 * 4 * 512))
test_features = np.reshape(test_features, (60, 4 * 4 * 512))

Define and train the densley connected classifier

In [0]:
from keras import models
from keras import layers
from keras import optimizers

model = models.Sequential()
model.add(layers.Dense(256, activation='relu', input_dim=4 * 4 * 512))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(1, activation='sigmoid'))

model.compile(optimizer=optimizers.RMSprop(lr=2e-5),
              loss='binary_crossentropy',
              metrics=['acc'])

histroy = model.fit(train_features, train_labels,
                    epochs=5,
                    batch_size=1,
                    validation_data=(validation_features, validation_labels))

model.summary()

In [0]:
import matplotlib.pyplot as plt

acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(1, len(acc) + 1)

plt.plot(epochs, acc, 'bo', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='validation acc')
plt.title('Training and Validation loss')
plt.legend()

plt.figure()

plt.plot(epochs, loss, 'bo', label='training loss')
plt.plot(epochs, val_loss, 'b', label='validation acc')
plt.title('Training and Validation loss')
plt.legend()

plt.show()

* implementing feature extraction with data augmentation
* add a densley connected classifier on top of the convolutional base


In [0]:
from keras import models
from keras import layers

model = models.Sequential()
model.add(conv_base)
model.add(layers.Flatten())
model.add(layers.Dense(256, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))

model.summary()

At this point we will need to freeze the network to prevent the weights from being updated during training.  If we don't freeze the weights, the previous learned representations will be destroyed.

In [0]:
print('This is the number of trainable weights '
      'before freezing the conv base:', len(model.trainable_weights))

conv_base.trainable = False

print('This is the number of trainable weights '
      'after freezing the conv base:', len(model.trainable_weights))

In [0]:
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers

train_datagen = ImageDataGenerator(
    rescale=1.255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest')


test_generator = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(150, 150),
    batch_size=5,
    class_mode='binary')

validation_generator = test_datagen.flow_from_directory(
    validation_dir,
    target_size=(150, 150),
    batch_size=1,
    class_mode='binary')

model.compile(loss='binary_crossentropy',
              optimizer=optimizers.RMSprop(lr=2e-5),
              metrics=['acc'])


history = model.fit_generator(
    train_generator,
    steps_per_epoch=100,
    epochs=5,
    validation_data=validation_generator,
    validation_steps=50)

Add fine tuning to to our model.  We will freeze some layers and unfreeze others

In [0]:
conv_base.trainable = True

set_trainable = False
for layer in conv_base.layers:
  if layer.name == 'block5_conv1':
    set_trainable = True
  if set_trainable:
    layer.trainable = True
  else:
    layer.trainable = False

To finish our fine tuning we will train the model using a very RMSprop low learning rate.

In [0]:
model.compile(loss='binary_crossentropy',
              optimizer=optimizers.RMSprop(lr=1e-5),
              metrics=['acc'])

history = model.fit_generator(
    train_generator,
    steps_per_epoch=100,
    epochs=5,
    validation_data=validation_generator,
    validation_steps=50)


Plotting the model

In [0]:
def smooth_curve(points, factor=0.8):
  smoothed_points = []
  for point in points:
    if smoothed_points:
     previous = smoothed_points[-1]
     smoothed_points.append(previous * factor + point * (1 - factor))
    else:
      smoothed_points.append(point)
  return smoothed_points

plt.plot(epochs,
         smooth_curve(acc), 'bo', label='Smoothed training acc')
plt.plot(epochs,
         smooth_curve(val_acc), 'b', label='Smoothed validation acc')
plt.title('Training and Validation accuracy')
plt.legend()

plt.figure()

plt.plot(epochs,
         smooth_curve(loss), 'bo', label='Smoothed training loss')
plt.title('Training and validation loss')
plt.legend()

plt.show()


Time to Evaluate the model we build

In [0]:
test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(150, 150),
    batch_size=1,
    class_mode='binary')

test_loss, test_acc = model.evaluate_generator(test_generator, steps=50)
print('test acc:', test_acc)